# Run the `snowBMI` model using Basic Model Interface commands

This notebook demonstrates the execution of a simple temperature index snow model using BMI. View the source code for the [model](https://github.com/SnowHydrology/snowBMI/blob/master/snow/snow.py) and its [BMI](https://github.com/SnowHydrology/snowBMI/blob/master/snow/bmi_snow.py) on GitHub.

This exercise is based on the [heat](https://github.com/csdms/bmi-example-python) example from [CSDMS](https://csdms.colorado.edu/wiki/Main_Page).

Start by importing `os`, `numpy` and the `Snow` BMI:

In [1]:
import os
import numpy as np

from snow import SnowBmi

Create an instance of the model's BMI.

In [2]:
x = SnowBmi()

Use the BMI `get_component_name` function to query the model's name.

In [3]:
print(x.get_component_name())

Temperature Index Snow Model with BMI


Start the `Snow` model through its BMI using a configuration file. First, take a look at the file and then run the BMI `initialize` function.

In [4]:
cat snow.yaml

# Snow model configuration
rs_method: 1         # 1 = single threshold, 2 = dual threshold
rs_thresh: 2.5       # rain-snow temperature threshold when rs_method = 1 (°C)
snow_thresh_max: 1.5 # maximum all-snow temp when rs_method = 2 (°C)
rain_thresh_min: 4.5 # minimum all-rain temp when rs_method = 2 (°C)
ddf_max: 1           # maximum degree day melt factor (mm/day/°C)
ddf_min: 0           # minimum degree day melt factor (mm/day/°C)
tair_melt_thresh: 1  # air temperature threshold above which melt can occur (°C)
swe_init: 0          # initial snow water equivalent (mm)
dayofyear: 274       # Day of year of simulation start (ex: 1 = Jan 1, 274 = Oct 1)

In [5]:
x.initialize("snow.yaml")

Check the time information for the model.

In [6]:
print("Start time:", x.get_start_time())
print("End time:", x.get_end_time())
print("Current time:", x.get_current_time())
print("Time step:", x.get_time_step())
print("Time units:", x.get_time_units())

Start time: 0.0
End time: 1.7976931348623157e+308
Current time: 0.0
Time step: 86400
Time units: s


Show the input and output variables for the component (aside on [Standard Names](https://csdms.colorado.edu/wiki/CSDMS_Standard_Names)):

In [7]:
print("Input vars =", x.get_input_var_names())
print("Output vars =", x.get_output_var_names())

Input vars = ('atmosphere_water__precipitation_leq-volume_flux', 'land_surface_air__temperature')
Output vars = ('snowpack__liquid-equivalent_depth', 'snowpack__melt_volume_flux')


For a quick example, set air temperature and precipitation to a constant value using standard BMI functions.

In [8]:
# when the model is initialized, air temperature equals 0
# we can check this using either get_value or get_value_ptr
temp_array = np.zeros(1,)
x.get_value('land_surface_air__temperature', temp_array)
print("Air temperature from get_value =", temp_array)
print("Air temperature from get_value_ptr =", x.get_value_ptr('land_surface_air__temperature'))

# Now we can set the value of air temperature
# first make a numpy single-element array and give it a value
air_temperature = np.full(1, -5)

# Then set the value and check it with get_value
x.set_value("land_surface_air__temperature", air_temperature)
x.get_value('land_surface_air__temperature', temp_array)
print("Air temperature from get_value after set_value=", temp_array)
print("Air temperature from get_value_ptr after set_value=", x.get_value_ptr('land_surface_air__temperature'))

# And let's do the same for precipitation
precip = np.full(1, 10)
print("Precipitation from get_value_ptr =", x.get_value_ptr("atmosphere_water__precipitation_leq-volume_flux"))
x.set_value("atmosphere_water__precipitation_leq-volume_flux", precip)
print("Precipitation from get_value_ptr after set_value=", x.get_value_ptr('atmosphere_water__precipitation_leq-volume_flux'))

Air temperature from get_value = [0.]
Air temperature from get_value_ptr = [0.]
Air temperature from get_value after set_value= [-5.]
Air temperature from get_value_ptr after set_value= [-5.]
Precipitation from get_value_ptr = [0.]
Precipitation from get_value_ptr after set_value= [10.]


A key advantage of BMI is that **we don't have to know the names of any of the input or output variables**. We can query the model and have standardized BMI functions return their names and values. This means no more spending hours poring over code to get the simplest info—you can use the same functions over and over again to get the info you require.

In [10]:
# we can also look at all values with a loop
input_vars = x.get_input_var_names()
for tmp in input_vars:
    print(tmp, "=", x.get_value_ptr(tmp))
output_vars = x.get_output_var_names()
for tmp in output_vars:
    print(tmp, "=", x.get_value_ptr(tmp))

atmosphere_water__precipitation_leq-volume_flux = [10.]
land_surface_air__temperature = [-5.]
snowpack__liquid-equivalent_depth = [0.]
snowpack__melt_volume_flux = [0.]


In [12]:
# update for a set number of time steps
run_steps = 20
for i in range(run_steps):
    x.update()
    print("swe =", x.get_value_ptr("snowpack__liquid-equivalent_depth"))

swe in model =  [210.]
precip in model = [10.]
snowfall in model = [10.]
air temp in model = [-5.]
rs_method in model = 1
swe = [210.]
swe in model =  [220.]
precip in model = [10.]
snowfall in model = [10.]
air temp in model = [-5.]
rs_method in model = 1
swe = [220.]
swe in model =  [230.]
precip in model = [10.]
snowfall in model = [10.]
air temp in model = [-5.]
rs_method in model = 1
swe = [230.]
swe in model =  [240.]
precip in model = [10.]
snowfall in model = [10.]
air temp in model = [-5.]
rs_method in model = 1
swe = [240.]
swe in model =  [250.]
precip in model = [10.]
snowfall in model = [10.]
air temp in model = [-5.]
rs_method in model = 1
swe = [250.]
swe in model =  [260.]
precip in model = [10.]
snowfall in model = [10.]
air temp in model = [-5.]
rs_method in model = 1
swe = [260.]
swe in model =  [270.]
precip in model = [10.]
snowfall in model = [10.]
air temp in model = [-5.]
rs_method in model = 1
swe = [270.]
swe in model =  [280.]
precip in model = [10.]
snowfall

In [13]:
# Try updating air temp to produce melt
air_temperature = np.full(1, 15)

# Then set the value and check it with get_value
x.set_value("land_surface_air__temperature", air_temperature)
print("Air temperature from get_value_ptr after set_value=", x.get_value_ptr('land_surface_air__temperature'))

# update for a set number of time steps
run_steps = 20
for i in range(run_steps):
    x.update()
    print("swe =", x.get_value_ptr("snowpack__liquid-equivalent_depth"))


Air temperature from get_value_ptr after set_value= [15.]
swe in model =  [398.34770504]
precip in model = [10.]
snowfall in model = [0.]
air temp in model = [15.]
rs_method in model = 1
swe = [398.34770504]
swe in model =  [396.77237017]
precip in model = [10.]
snowfall in model = [0.]
air temp in model = [15.]
rs_method in model = 1
swe = [396.77237017]
swe in model =  [395.27238786]
precip in model = [10.]
snowfall in model = [0.]
air temp in model = [15.]
rs_method in model = 1
swe = [395.27238786]
swe in model =  [393.84612824]
precip in model = [10.]
snowfall in model = [0.]
air temp in model = [15.]
rs_method in model = 1
swe = [393.84612824]
swe in model =  [392.49193961]
precip in model = [10.]
snowfall in model = [0.]
air temp in model = [15.]
rs_method in model = 1
swe = [392.49193961]
swe in model =  [391.20814891]
precip in model = [10.]
snowfall in model = [0.]
air temp in model = [15.]
rs_method in model = 1
swe = [391.20814891]
swe in model =  [389.99306219]
precip in m

In [11]:
print(x._values)
print(x._model._dayofyear)
print(x._model._tair_c)
test = x.get_value_ptr("snowpack__melt_volume_flux")
print(test)
print(test.shape)

{'atmosphere_water__precipitation_leq-volume_flux': array([10.]), 'land_surface_air__temperature': array([-5.]), 'snowpack__liquid-equivalent_depth': array([0.]), 'snowpack__melt_volume_flux': array([0.])}
294
[-5.]
[0.]
(1,)


Next, get the identifier for the grid on which the temperature variable is defined:

In [ ]:
grid_id = x.get_var_grid("plate_surface__temperature")
print("Grid id:", grid_id)

Then get the grid attributes:

In [ ]:
print("Grid type:", x.get_grid_type(grid_id))

rank = x.get_grid_rank(grid_id)
print("Grid rank:", rank)

shape = np.ndarray(rank, dtype=int)
x.get_grid_shape(grid_id, shape)
print("Grid shape:", shape)

spacing = np.ndarray(rank, dtype=float)
x.get_grid_spacing(grid_id, spacing)
print("Grid spacing:", spacing)

These commands are made somewhat un-Pythonic by the generic design of the BMI.

Through the model's BMI, zero out the initial temperature field, except for an impulse near the middle.
Note that *set_value* expects a one-dimensional array for input.

In [ ]:
temperature = np.zeros(shape)
temperature[3, 4] = 100.0
x.set_value("plate_surface__temperature", temperature)

Check that the temperature field has been updated. Note that *get_value* expects a one-dimensional array to receive output.

In [12]:
temperature_flat = np.empty_like(temperature).flatten()
x.get_value("plate_surface__temperature", temperature_flat)
print(temperature_flat.reshape(shape))

AttributeError: 'int' object has no attribute 'flatten'

Now advance the model by a single time step:

In [ ]:
x.update()

View the new state of the temperature field:

In [ ]:
x.get_value("plate_surface__temperature", temperature_flat)
print(temperature_flat.reshape(shape))

There's diffusion!

Advance the model to some distant time:

In [ ]:
distant_time = 2.0
while x.get_current_time() < distant_time:
    x.update()

View the final state of the temperature field:

In [ ]:
np.set_printoptions(formatter={"float": "{: 5.1f}".format})
x.get_value("plate_surface__temperature", temperature_flat)
print(temperature_flat.reshape(shape))

Note that temperature isn't conserved on the plate:

In [ ]:
print(temperature_flat.sum())

End the model:

In [ ]:
x.finalize()